In [11]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
plt.style.use("seaborn-v0_8-darkgrid")

In [13]:
import pyarrow
print(pyarrow.__version__)

22.0.0


In [14]:
import sys
print(sys.executable)

/workspaces/Aadhaar-Friction-Index/venv/bin/python


## Output paths

In [15]:
from pathlib import Path

PLOT_PATH = Path("../outputs/plots")
TABLE_PATH = Path("../outputs/tables")

PLOT_PATH.mkdir(parents=True, exist_ok=True)
TABLE_PATH.mkdir(parents=True, exist_ok=True)



In [16]:
print(type(PLOT_PATH), PLOT_PATH)

<class 'pathlib.PosixPath'> ../outputs/plots


## Load Data

In [17]:
afi_df = pd.read_csv(
    "../datasets/processed/index/aadhaar_friction_index_only.csv"
)

signals_df = pd.read_csv(
    "../datasets/processed/signals/friction_signals.csv"
)

# Clean period
afi_df = afi_df.dropna(subset=["period"])
signals_df = signals_df.dropna(subset=["period"])


## AFI Heatmap (District × Time)

- Fixes ugly heatmap
- Uses top 25 districts only (readable)

In [18]:
top_districts = (
    afi_df.groupby("district")["AFI"]
    .mean()
    .sort_values(ascending=False)
    .head(25)
    .index
)

heatmap_df = afi_df[
    afi_df["district"].isin(top_districts)
]

pivot_df = heatmap_df.pivot_table(
    index="district",
    columns="period",
    values="AFI",
    aggfunc="mean"
)

plt.figure(figsize=(14, 10))
sns.heatmap(
    pivot_df,
    cmap="Reds",
    linewidths=0.3,
    linecolor="gray"
)

plt.title("Aadhaar Friction Index Heatmap (Top 25 Districts)")
plt.xlabel("Time (Month)")
plt.ylabel("District")

plt.tight_layout()
plt.savefig(PLOT_PATH /"afi_heatmap_district_time.png", dpi=300, bbox_inches="tight")
plt.close()


## Lifecycle Flow Imbalance Chart

(Enrolment vs Updates)

In [19]:
flow_df = signals_df.groupby("period").agg({
    "enrolment_count": "sum",
    "demographic_update_count": "sum",
    "biometric_update_count": "sum"
}).reset_index()

flow_df["total_updates"] = (
    flow_df["demographic_update_count"] +
    flow_df["biometric_update_count"]
)

plt.figure(figsize=(12, 6))
plt.plot(flow_df["period"], flow_df["enrolment_count"], label="Enrolments")
plt.plot(flow_df["period"], flow_df["total_updates"], label="Total Updates")

plt.xticks(rotation=45)
plt.title("Lifecycle Flow Imbalance: Enrolment vs Updates")
plt.xlabel("Period")
plt.ylabel("Count")
plt.legend()

plt.tight_layout()
plt.savefig(PLOT_PATH / "lifecycle_flow_imbalance.png", dpi=300)
plt.close()


## District Typology (TABLE Output)

In [20]:
district_avg = (
    afi_df.groupby(["state", "district"])["AFI"]
    .mean()
    .reset_index()
)

def classify_friction(afi):
    if afi >= 70:
        return "High Friction"
    elif afi >= 40:
        return "Medium Friction"
    else:
        return "Low Friction"

district_avg["friction_type"] = district_avg["AFI"].apply(classify_friction)

district_avg.to_csv(
    TABLE_PATH / "district_friction_typology.csv",
    index=False
)


## Before / After AFI Trend (Selected Districts)

In [21]:
selected_districts = (
    afi_df.groupby("district")["AFI"]
    .mean()
    .sort_values(ascending=False)
    .head(5)
    .index
)

trend_df = afi_df[afi_df["district"].isin(selected_districts)]

plt.figure(figsize=(12, 6))

for district in selected_districts:
    d = trend_df[trend_df["district"] == district]
    plt.plot(d["period"], d["AFI"], label=district)

plt.xticks(rotation=45)
plt.title("AFI Trend for High-Friction Districts")
plt.xlabel("Period")
plt.ylabel("AFI Score")
plt.legend()

plt.tight_layout()
plt.savefig(PLOT_PATH / "afi_trend_selected_districts.png", dpi=300)
plt.close()


## Hidden Risk Scatter Plot

(Low enrolment updates ≠ low friction)

In [22]:
risk_df = signals_df.merge(
    afi_df,
    on=["state", "district", "period"],
    how="inner"
)

risk_df["update_intensity"] = (
    risk_df["demographic_update_count"] +
    risk_df["biometric_update_count"]
)

plt.figure(figsize=(10, 6))
plt.scatter(
    risk_df["update_intensity"],
    risk_df["AFI"],
    alpha=0.5
)

plt.title("Hidden Risk: Low Updates ≠ Low Friction")
plt.xlabel("Total Update Volume")
plt.ylabel("AFI Score")

plt.tight_layout()
plt.savefig(PLOT_PATH / "hidden_risk_scatter.png", dpi=300)
plt.close()


## AFI Summary by District (CORE TABLE)

Purpose:
→ “Which districts are structurally high friction?”

In [23]:
district_summary = (
    afi_df
    .groupby(["state", "district"])
    .agg(
        avg_afi=("AFI", "mean"),
        max_afi=("AFI", "max"),
        min_afi=("AFI", "min"),
        months_observed=("period", "nunique")
    )
    .reset_index()
    .sort_values("avg_afi", ascending=False)
)

district_summary.to_csv(
    TABLE_PATH / "afi_summary_by_district.csv",
    index=False
)


## AFI Summary by State (Policy View)

Purpose:
→ State-wise prioritization & funding logic

In [24]:
state_summary = (
    afi_df
    .groupby("state")
    .agg(
        avg_afi=("AFI", "mean"),
        max_afi=("AFI", "max"),
        districts=("district", "nunique")
    )
    .reset_index()
    .sort_values("avg_afi", ascending=False)
)

state_summary.to_csv(
    TABLE_PATH / "afi_summary_by_state.csv",
    index=False
)


## Monthly AFI Trends (Time-Series Table)

Purpose:
→ Used directly in Power BI / Excel line charts

In [25]:
monthly_afi = (
    afi_df
    .groupby("period")
    .agg(
        avg_afi=("AFI", "mean"),
        median_afi=("AFI", "median"),
        districts_reporting=("district", "nunique")
    )
    .reset_index()
)

monthly_afi.to_csv(
    TABLE_PATH / "monthly_afi_trends.csv",
    index=False
)


## Friction Signal Summary (Explainability Table)

Purpose:
→ “WHY is AFI high here?”

In [26]:
signal_summary = (
    signals_df
    .groupby(["state", "district"])
    .agg(
        avg_UIS=("UIS", "mean"),
        avg_RIS=("RIS", "mean"),
        avg_BSS=("BSS", "mean"),
        avg_TSD=("TSD", "mean"),
        months=("period", "nunique")
    )
    .reset_index()
)

signal_summary.to_csv(
    TABLE_PATH / "friction_signal_summary.csv",
    index=False
)


## Lifecycle Imbalance Table (NOT a plot)

Purpose:
→ Quantifies enrolment vs update pressure

In [27]:
lifecycle_table = (
    signals_df
    .groupby("period")
    .agg(
        total_enrolments=("enrolment_count", "sum"),
        total_demographic_updates=("demographic_update_count", "sum"),
        total_biometric_updates=("biometric_update_count", "sum")
    )
    .reset_index()
)

lifecycle_table["total_updates"] = (
    lifecycle_table["total_demographic_updates"] +
    lifecycle_table["total_biometric_updates"]
)

lifecycle_table["update_to_enrolment_ratio"] = (
    lifecycle_table["total_updates"] /
    lifecycle_table["total_enrolments"].replace(0, pd.NA)
)

lifecycle_table.to_csv(
    TABLE_PATH / "lifecycle_imbalance_table.csv",
    index=False
)


## Hidden Risk Table (VERY IMPORTANT)

Purpose:
→ Finds districts with low volume but high friction

In [28]:
risk_table = signals_df.merge(
    afi_df,
    on=["state", "district", "period"],
    how="inner"
)

risk_table["total_updates"] = (
    risk_table["demographic_update_count"] +
    risk_table["biometric_update_count"]
)

hidden_risk = risk_table[
    (risk_table["total_updates"] < risk_table["total_updates"].median()) &
    (risk_table["AFI"] > risk_table["AFI"].quantile(0.75))
]

hidden_risk = hidden_risk[
    ["state", "district", "period", "total_updates", "AFI"]
].sort_values("AFI", ascending=False)

hidden_risk.to_csv(
    TABLE_PATH / "hidden_risk_table.csv",
    index=False
)


## Top 100 High Friction Records (Audit Table)

Purpose:
→ Manual inspection / case studies

In [29]:
top_100 = (
    afi_df
    .sort_values("AFI", ascending=False)
    .head(100)
)

top_100.to_csv(
    TABLE_PATH / "top_100_high_friction_records.csv",
    index=False
)


## Paths 
- Ensure everything is human-readable tabular

In [36]:
from pathlib import Path
import sqlite3

BASE_TABLE_PATH = Path("../outputs/tables")

PARQUET_PATH = BASE_TABLE_PATH / "parquet"
EXCEL_PATH = BASE_TABLE_PATH / "excel"
SQLITE_PATH = BASE_TABLE_PATH / "sqlite"

for p in [PARQUET_PATH, EXCEL_PATH, SQLITE_PATH]:
    p.mkdir(parents=True, exist_ok=True)


## Load EXISTING CSVs

In [31]:
district_summary = pd.read_csv(TABLE_PATH / "afi_summary_by_district.csv")
state_summary = pd.read_csv(TABLE_PATH / "afi_summary_by_state.csv")
signal_summary = pd.read_csv(TABLE_PATH / "friction_signal_summary.csv")
hidden_risk = pd.read_csv(TABLE_PATH / "hidden_risk_table.csv")

## 1. PARQUET
(Only run if pyarrow is installed)

In [32]:
district_summary.to_parquet(
    PARQUET_PATH / "afi_summary_by_district.parquet",
    index=False
)

state_summary.to_parquet(
    PARQUET_PATH / "afi_summary_by_state.parquet",
    index=False
)

signal_summary.to_parquet(
    PARQUET_PATH / "friction_signal_summary.parquet",
    index=False
)

hidden_risk.to_parquet(
    PARQUET_PATH / "hidden_risk.parquet",
    index=False
)


## 2. SQLITE

In [34]:
conn = sqlite3.connect(
    SQLITE_PATH / "aadhaar_friction_tables.db"
)

district_summary.to_sql(
    "afi_by_district",
    conn,
    if_exists="replace",
    index=False
)

state_summary.to_sql(
    "afi_by_state",
    conn,
    if_exists="replace",
    index=False
)

signal_summary.to_sql(
    "friction_signal_summary",
    conn,
    if_exists="replace",
    index=False
)

hidden_risk.to_sql(
    "hidden_risk",
    conn,
    if_exists="replace",
    index=False
)

conn.close()


## 3. EXCEL

In [35]:
with pd.ExcelWriter(
    EXCEL_PATH / "afi_analysis_tables.xlsx",
    engine="xlsxwriter"
) as writer:

    district_summary.to_excel(
        writer,
        sheet_name="AFI by District",
        index=False
    )

    state_summary.to_excel(
        writer,
        sheet_name="AFI by State",
        index=False
    )

    signal_summary.to_excel(
        writer,
        sheet_name="Friction Signals",
        index=False
    )

    hidden_risk.to_excel(
        writer,
        sheet_name="Hidden Risk Districts",
        index=False
    )
